In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import copy
import time

In [2]:
NEW_NUM_DIST = np.array([2] * 9 + [4])
INIT_NUM_CELLS = 2
MOVES = ["h", "j", "k", "l"]
R_INVALID_MOVE = -1

---

In [3]:
class game():
    def __init__(self, N):
        """ make an NxN array and put numbers in two cells """
        self.N = N
        self.board = np.zeros((N, N))
        self.score = 0
        for _ in range(INIT_NUM_CELLS):
            self.add_num()
        print(self.board)

    def add_num(self):
        """
        self.board is a numpy array representing the game board
        add a random number according to NEW_NUM_DIST to one of
        the cells that are zero
        first need to check that there are at least one cell equal to zero
        """
        zero_inds = np.where(self.board == 0)
        num_zero_cells = zero_inds[0].size
        assert(num_zero_cells > 0)
        i = np.random.randint(num_zero_cells)
        self.board[zero_inds[0][i], zero_inds[1][i]] = np.random.choice(NEW_NUM_DIST)
        
    def left(self):
        """ change self.board to the state after a left move """
        for i in range(self.N):
            self.left_row(i)

    def right(self):
        """ change self.board to the state after a right move """
        for i in range(self.N):
            self.board[i] = self.board[i, ::-1]
            self.left_row(i)
            self.board[i] = self.board[i, ::-1]
    
    def up(self):
        """ change the board to the state after an up move """
        self.board = self.board.T
        self.left()
        self.board = self.board.T
        
    def down(self):
        """ change the board to the state after a down move """
        self.board = self.board.T
        self.right()
        self.board = self.board.T

    def left_row(self, i):
        """
        row is a 1d np array, possibly with zeros
        return a row with the state after a left move,
        which will merge cells with the same numbers
        """
        row = self.board[i]
        old_row = [x for x in row if x != 0]
        new_row, added_score = merge(old_row)
        new_row_array = np.zeros(row.size)
        new_row_array[:len(new_row)] = new_row
        self.board[i] = new_row_array
        self.score += added_score
    
    def can_move(self):
        """
        return true if there are zeros in the board
        or there are adjacent cells that are the same
        """
        for i in range(self.N):
            for j in range(self.N):
                # if any cell is equal to one of its neighbors then True
                if i - 1 >= 0:
                    if self.board[i - 1, j] == self.board[i, j]:
                        return True
                if i + 1 < self.N:
                    if self.board[i + 1, j] == self.board[i, j]:
                        return True
                if j - 1 >= 0:
                    if self.board[i, j - 1] == self.board[i, j]:
                        return True
                if j + 1 < self.N:
                    if self.board[i, j + 1] == self.board[i, j]:
                        return True
        if np.argwhere(self.board == 0).size != 0:
            # if there are any empty cells then True
            return True
        return False
    
    def change_state(self, key_in):
        """
        change the board to the state after the move
        and update the score accordingly
        """
        old_board = np.copy(self.board)
        old_score = self.score
        if key_in == "h":
            self.left()
        elif key_in == "j":
            self.down()
        elif key_in == "k":
            self.up()
        elif key_in == "l":
            self.right()
        else:
            pass
        if (self.board != old_board).any():
            # if the board changed, then add a new number to an empty cell. note that
            # the board might not change at all if we attemp to move in one direction
            # even though the board *can* be moved in some other direction
            self.add_num()
        return self.score - old_score

    def play_game(self):
        """
        ask for move, change state accordingly and print score and board, util
        no moves are allowed
        """
        i = 0
        while self.can_move():
            print(self.board, "score = ", self.score)
#             print("i = ", i)
            key_in = input("Enter direction (h, j, k, l): ")
            _ = self.change_state(key_in)
        print(self.board, "score = ", self.score)
        
    def board_str(self):
        """ convert the board to a unique string representation """
        return "".join([str(int(i)) for i in self.board.flatten()])
    
    def valid_action(self, key_in):
        """
        return True if the move can change the state of the board, otherwise
        False. use this for various search algorithms where we need to know
        if an action is allowed or not
        """
        g_copy = copy.deepcopy(self)
        _ = g_copy.change_state(key_in)
        return (g_copy.board != self.board).any()
    
    def get_valid_actions(self):
        valid_actions = []
        for move in MOVES:
            if self.valid_action(move):
                valid_actions.append(move)
        return valid_actions

def merge(old_row):
    """
    old_row and is a 1d list with no zeros
    new_row is the state after the left move,
    which will merge cells with the same numbers
    """
    new_row = []
    added_score = 0
    while len(old_row) >= 2:
        # if there are still at least two numbers in the old_row,
        # then try to merge the 0th and 1st numbers
        if old_row[0] == old_row[1]:
            new_row.append(old_row[0] * 2)
            added_score += old_row[0] * 2
            old_row = old_row[2:]
        else:
            new_row.append(old_row[0])
            old_row = old_row[1:]
    # len(old_row) should be either 0 or 1
    assert(len(old_row) <= 1)
    if len(old_row) == 1:
        new_row.append(old_row[0])
    return new_row, added_score

In [4]:
def select_action(g, ites):
    valid_actions = g.get_valid_actions()
    actions_score = np.zeros(len(MOVES))
    for i, move in enumerate(MOVES):
        if move in valid_actions:
            for _ in range(ites):
                g_copy = copy.deepcopy(g)
                old_score = g_copy.score
                g_copy.change_state(move)
                while g_copy.can_move():
                    move2 = np.random.choice(g_copy.get_valid_actions())
                    g_copy.change_state(move2)
                actions_score[i] += g_copy.score - old_score
            actions_score[i] = actions_score[i] / ites
        else:
            actions_score[i] = 0
    return MOVES[np.argmax(actions_score)]

In [8]:
def select_action1(g, ites):
    valid_actions = g.get_valid_actions()
    actions_score = np.zeros(len(MOVES))
    for i, move in enumerate(MOVES):
        if move in valid_actions:
            for _ in range(ites):
                g_copy = copy.deepcopy(g)
                old_score = g_copy.score
                g_copy.change_state(move)
                while g_copy.can_move():
                    move2 = np.random.choice(g_copy.get_valid_actions())
                    g_copy.change_state(move2)
                actions_score[i] += g_copy.board.max()
            actions_score[i] = actions_score[i] / ites
        else:
            actions_score[i] = 0
    return MOVES[np.argmax(actions_score)]

In [9]:
def select_action2(g, ites):
    valid_actions = g.get_valid_actions()
    actions_score = np.zeros(len(MOVES))
    for i, move in enumerate(MOVES):
        if move in valid_actions:
            for _ in range(ites):
                g_copy = copy.deepcopy(g)
                old_score = g_copy.score
                g_copy.change_state(move)
                while g_copy.can_move():
                    move2 = np.random.choice(g_copy.get_valid_actions())
                    g_copy.change_state(move2)
                actions_score[i] += g_copy.board.sum()
            actions_score[i] = actions_score[i] / ites
        else:
            actions_score[i] = 0
    return MOVES[np.argmax(actions_score)]

In [13]:
def pure_mcts_play(g, ites=50):
    i = 0
    while g.can_move():
        print(i)
        print(g.board, "score = ", g.score)
        move = select_action2(g, ites=ites)
        print(move)
        _ = g.change_state(move)
        i += 1
        print()
    print(g.board, "score = ", g.score)

---

Different end scores

In [12]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g, ites=10)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
h

1
[[0. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  0
h

2
[[2. 0. 0. 2.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  0
h

3
[[4. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  4.0
j

4
[[2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [8. 0. 0. 0.]
 [4. 0. 0. 0.]] score =  16.0
k

5
[[2. 0. 0. 0.]
 [8. 0. 2. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  16.0
l

6
[[0. 2. 0. 2.]
 [0. 0. 8. 2.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]] score =  16.0
k

7
[[0. 2. 8. 4.]
 [0. 0. 0. 4.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  20.0
h

8
[[2. 8. 4. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 2. 0. 0.]] score =  20.0
l

9
[[0. 2. 8. 4.]
 [2. 0. 0. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 2.]] score =  20.0
j

10
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 8.]
 [2. 2. 8. 4.]] score =  32.0
h

11
[[0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 8. 0. 0.]
 [4. 8. 4. 0.]] scor

In [14]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g, ites=10)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
h

1
[[0. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  0
h

2
[[2. 0. 0. 2.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  0
h

3
[[4. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  4.0
j

4
[[2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [8. 0. 0. 0.]
 [4. 0. 0. 0.]] score =  16.0
k

5
[[2. 0. 0. 0.]
 [8. 0. 2. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  16.0
l

6
[[0. 2. 0. 2.]
 [0. 0. 8. 2.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]] score =  16.0
k

7
[[0. 2. 8. 4.]
 [0. 0. 0. 4.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  20.0
h

8
[[2. 8. 4. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 2. 0. 0.]] score =  20.0
l

9
[[0. 2. 8. 4.]
 [2. 0. 0. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 2.]] score =  20.0
j

10
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 8.]
 [2. 2. 8. 4.]] score =  32.0
h

11
[[0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 8. 0. 0.]
 [4. 8. 4. 0.]] scor

---

In [12]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g, ites=10)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
h

1
[[0. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  0
h

2
[[2. 0. 0. 2.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  0
h

3
[[4. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  4.0
j

4
[[2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [8. 0. 0. 0.]
 [4. 0. 0. 0.]] score =  16.0
k

5
[[2. 0. 0. 0.]
 [8. 0. 2. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  16.0
l

6
[[0. 2. 0. 2.]
 [0. 0. 8. 2.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]] score =  16.0
k

7
[[0. 2. 8. 4.]
 [0. 0. 0. 4.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  20.0
h

8
[[2. 8. 4. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 2. 0. 0.]] score =  20.0
l

9
[[0. 2. 8. 4.]
 [2. 0. 0. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 2.]] score =  20.0
j

10
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 8.]
 [2. 2. 8. 4.]] score =  32.0
h

11
[[0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 8. 0. 0.]
 [4. 8. 4. 0.]] scor

In [7]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g, ites=20)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
j

1
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 2. 4.]
 [0. 0. 0. 2.]] score =  0
k

2
[[0. 0. 2. 4.]
 [2. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  0
j

3
[[0. 0. 4. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [2. 0. 2. 2.]] score =  0
l

4
[[0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 2. 2. 4.]] score =  4.0
k

5
[[0. 2. 2. 8.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 2. 0. 0.]] score =  12.0
j

6
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 8.]
 [0. 4. 2. 4.]] score =  16.0
k

7
[[0. 2. 2. 8.]
 [0. 4. 0. 4.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  16.0
l

8
[[0. 0. 4. 8.]
 [0. 0. 0. 8.]
 [2. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  28.0
h

9
[[4. 8. 0. 0.]
 [8. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 2.]] score =  28.0
l

10
[[0. 0. 4. 8.]
 [0. 0. 0. 8.]
 [0. 0. 0. 2.]
 [2. 0. 0. 4.]] score =  32.0
j

11
[[ 0.  0.  0.  0.]
 [ 0.  0.  0. 16.]
 [ 0.  0.  0.  2.]
 [ 2.  2.  

In [14]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g, ites=30)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
j

1
[[2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 2.]] score =  0
k

2
[[2. 0. 0. 4.]
 [2. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  0
j

3
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [4. 2. 0. 2.]] score =  4.0
k

4
[[4. 2. 0. 4.]
 [0. 2. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  4.0
j

5
[[0. 0. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 4.]
 [4. 4. 0. 2.]] score =  8.0
k

6
[[4. 4. 2. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]
 [4. 0. 0. 0.]] score =  8.0
h

7
[[8. 2. 4. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [4. 0. 0. 0.]] score =  16.0
l

8
[[0. 8. 2. 4.]
 [0. 0. 2. 2.]
 [0. 0. 0. 2.]
 [0. 0. 0. 4.]] score =  16.0
l

9
[[2. 8. 2. 4.]
 [0. 0. 0. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 4.]] score =  20.0
h

10
[[2. 8. 2. 4.]
 [4. 0. 0. 2.]
 [2. 0. 0. 0.]
 [4. 0. 0. 0.]] score =  20.0
l

11
[[2. 8. 2. 4.]
 [0. 2. 4. 2.]
 [0. 0. 0. 2.]
 [0. 0. 0. 4.]] score =

In [11]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g, ites=40)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
k

1
[[0. 0. 2. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  0
h

2
[[2. 4. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]] score =  0
k

3
[[4. 4. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]] score =  4.0
j

4
[[0. 0. 2. 0.]
 [0. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 4. 0. 4.]] score =  4.0
j

5
[[0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 4. 2. 4.]] score =  4.0
l

6
[[0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 2. 0. 4.]
 [2. 4. 2. 4.]] score =  4.0
l

7
[[0. 0. 0. 4.]
 [0. 0. 0. 2.]
 [0. 0. 2. 4.]
 [2. 4. 2. 4.]] score =  4.0
j

8
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [2. 0. 0. 2.]
 [2. 4. 4. 8.]] score =  16.0
j

9
[[0. 0. 0. 2.]
 [0. 0. 0. 4.]
 [0. 0. 0. 2.]
 [4. 4. 4. 8.]] score =  20.0
h

10
[[2. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [8. 4. 8. 2.]] score =  28.0
k

11
[[2. 4. 8. 2.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]
 [8. 0. 2. 0.]] score = 

In [15]:
np.random.seed(2382282283)
g = game(4)
s = time.time()
pure_mcts_play(g)
print("time = ", time.time() - s)

[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]]
0
[[0. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
k

1
[[0. 0. 0. 4.]
 [0. 2. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  0
k

2
[[0. 2. 0. 4.]
 [0. 0. 0. 2.]
 [0. 2. 0. 0.]
 [0. 0. 0. 0.]] score =  0
h

3
[[2. 4. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 2.]] score =  0
l

4
[[0. 2. 2. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 2.]
 [0. 0. 0. 2.]] score =  0
k

5
[[0. 2. 2. 4.]
 [0. 0. 0. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 2.]] score =  4.0
k

6
[[0. 2. 2. 8.]
 [0. 2. 0. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] score =  16.0
h

7
[[4. 8. 0. 0.]
 [2. 4. 0. 0.]
 [0. 0. 0. 0.]
 [0. 2. 0. 0.]] score =  20.0
l

8
[[0. 0. 4. 8.]
 [0. 0. 2. 4.]
 [0. 0. 0. 0.]
 [0. 2. 0. 2.]] score =  20.0
k

9
[[0. 2. 4. 8.]
 [2. 0. 2. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]] score =  20.0
k

10
[[2. 2. 4. 8.]
 [0. 0. 2. 4.]
 [0. 0. 2. 2.]
 [0. 0. 0. 0.]] score =  20.0
h

11
[[4. 4. 8. 0.]
 [2. 4. 0. 0.]
 [4. 2. 0. 0.]
 [0. 0. 0. 0.]] score =  2